In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src import utils, limpeza_dados

ModuleNotFoundError: No module named 'src'

In [ ]:
df = utils.carregar_dados('../dados/brutos/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

In [ ]:
df = utils.carregar_dados('../dados/brutos/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()